# Convolutions en Vision par Ordinateur

Ce notebook explore les opérations de convolution et leur application dans les réseaux de neurones convolutifs (CNN).

## Plan du notebook :
1. Introduction aux convolutions
2. Implémentation manuelle des convolutions
3. Application de filtres sur des images MNIST
4. Couches convolutives avec PyTorch
5. Entraînement d'une couche convolutive sur des données synthétiques

In [1]:
# Import des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

# Configuration matplotlib pour de meilleurs affichages
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'

## 1. Introduction aux Convolutions

La convolution est une opération mathématique fondamentale en vision par ordinateur. Elle combine une image avec un filtre (noyau) pour extraire des caractéristiques spécifiques.

Formellement, la convolution 2D est définie par :

$(f * k)(x,y) = \sum_{i=-a}^a \sum_{j=-b}^b f(x-i,y-j)k(i,j)$

où $f$ est l'image d'entrée et $k$ est le noyau de convolution.

## 2. Implémentation Manuelle des Convolutions

Commençons par implémenter manuellement l'opération de convolution 2D.

In [ ]:
def manual_conv2d(image, kernel):
    """Calcule manuellement la convolution 2D."""
    # Récupérer les dimensions
    i_height, i_width = image.shape
    k_height, k_width = kernel.shape
    
    # Calculer le padding nécessaire
    pad_h = k_height // 2
    pad_w = k_width // 2
    
    # Créer la matrice de sortie
    output = np.zeros_like(image, dtype=float)
    
    # Padding de l'image d'entrée
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    # Effectuer la convolution
    for i in range(i_height):
        for j in range(i_width):
            # Extraire la région
            region = padded_image[i:i+k_height, j:j+k_width]
            # Calculer la convolution pour cette position
            output[i, j] = np.sum(region * kernel)
    
    return output

### Visualisation du Processus de Convolution

Visualisons étape par étape comment fonctionne une convolution sur un exemple simple.

In [ ]:
# Créer une matrice d'entrée simple (5x5)
input_matrix = np.zeros((5, 5))
input_matrix[1:4, 1:4] = 1  # Carré blanc au centre

# Définir un noyau de détection de contours vertical (3x3)
kernel = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

# Appliquer la convolution
result = manual_conv2d(input_matrix, kernel)

# Visualiser l'entrée, le noyau et le résultat
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(input_matrix)
axes[0].set_title("Image d'entrée")
axes[0].axis('off')

axes[1].imshow(kernel)
axes[1].set_title("Noyau (Sobel vertical)")
axes[1].axis('off')

axes[2].imshow(result)
axes[2].set_title("Résultat de la convolution")
axes[2].axis('off')

plt.tight_layout()
plt.show()

### Illustration de la Convolution sur une Position Spécifique

Voyons en détail comment une valeur spécifique est calculée lors de la convolution.

In [ ]:
# Fonction pour visualiser le calcul d'un pixel spécifique
def visualize_convolution_step(input_matrix, kernel, position):
    # Extraire les dimensions
    k_height, k_width = kernel.shape
    pad_h, pad_w = k_height // 2, k_width // 2
    
    # Padding de l'image
    padded = np.pad(input_matrix, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    # Position dans l'image paddée
    i, j = position
    padded_i, padded_j = i + pad_h, j + pad_w
    
    # Extraire la région pour la convolution
    region = padded[padded_i-pad_h:padded_i+pad_h+1, padded_j-pad_w:padded_j+pad_w+1]
    
    # Calcul de la valeur
    result_value = np.sum(region * kernel)
    
    # Visualisation
    fig, axes = plt.subplots(1, 4, figsize=(18, 5))
    
    # Image originale avec la position marquée
    axes[0].imshow(input_matrix)
    axes[0].plot(j, i, 'ro', markersize=10)
    axes[0].set_title("Position de calcul (rouge)")
    
    # Région extraite
    axes[1].imshow(region)
    axes[1].set_title("Région extraite")
    
    # Noyau
    axes[2].imshow(kernel)
    axes[2].set_title("Noyau")
    
    # Calcul détaillé
    axes[3].axis('off')
    calculation = f"Calcul de la convolution:\n\n"
    calculation += f"Région:\n{region}\n\n"
    calculation += f"Noyau:\n{kernel}\n\n"
    calculation += f"Produit élément par élément:\n{region * kernel}\n\n"
    calculation += f"Somme: {result_value}"
    axes[3].text(0.1, 0.5, calculation, fontsize=12, family='monospace', verticalalignment='center')
    
    plt.tight_layout()
    return result_value

# Visualiser le calcul pour une position au bord du carré
result_value = visualize_convolution_step(input_matrix, kernel, (1, 2))
print(f"Valeur calculée à la position (1, 2): {result_value}")

## 3. Application de Filtres sur des Images MNIST

Téléchargeons le dataset MNIST et appliquons différents filtres de convolution sur ces images.

In [ ]:
# Télécharger MNIST
mnist_data = torchvision.datasets.MNIST(root='./data', train=True, download=True)
print(f"Données MNIST chargées: {len(mnist_data)} images")

In [ ]:
# Sélectionner quelques exemples
sample_indices = [0, 1, 2, 3]  # Indices des images à afficher
samples = [mnist_data.data[i].numpy() for i in sample_indices]
labels = [mnist_data.targets[i].item() for i in sample_indices]

# Afficher les exemples
fig, axes = plt.subplots(1, len(samples), figsize=(15, 3))
for i, (img, lbl) in enumerate(zip(samples, labels)):
    axes[i].imshow(img)
    axes[i].set_title(f"Chiffre: {lbl}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

### Définition et Application de Différents Filtres

Appliquons différents noyaux de convolution sur une image MNIST.

In [ ]:
# Choisir une image de test
test_digit = samples[0].astype(np.float32)

# Définir différents noyaux
kernels = {
    "Sobel vertical": np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32),
    "Sobel horizontal": np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32),
    "Flou (moyenne)": np.ones((3, 3), dtype=np.float32) / 9,
    "Laplacien": np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32),
    "Accentuation": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
}

# Appliquer chaque filtre
results = {name: manual_conv2d(test_digit, kernel) for name, kernel in kernels.items()}

# Afficher les résultats
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Image originale
axes[0].imshow(test_digit)
axes[0].set_title("Image originale")
axes[0].axis('off')

# Images filtrées
for i, (name, result) in enumerate(results.items(), 1):
    axes[i].imshow(result)
    axes[i].set_title(name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 4. Convolutions avec PyTorch

Maintenant, utilisons PyTorch pour effectuer des convolutions et comparons les résultats avec notre implémentation manuelle.

In [ ]:
# Convertir l'image et le noyau en tenseurs PyTorch
def torch_convolution(image, kernel):
    """Applique une convolution en utilisant PyTorch."""
    # Convertir en tenseurs
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, H, W]
    kernel_tensor = torch.from_numpy(kernel).float().unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, K_H, K_W]
    
    # Appliquer la convolution
    result = F.conv2d(image_tensor, kernel_tensor, padding='same')
    
    # Convertir le résultat en numpy
    return result.squeeze().numpy()

# Comparer les résultats de notre implémentation avec PyTorch
kernel_name = "Sobel vertical"
kernel = kernels[kernel_name]

manual_result = manual_conv2d(test_digit, kernel)
torch_result = torch_convolution(test_digit, kernel)

# Afficher la comparaison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_digit)
axes[0].set_title("Image originale")
axes[0].axis('off')

axes[1].imshow(manual_result)
axes[1].set_title(f"Convolution manuelle ({kernel_name})")
axes[1].axis('off')

axes[2].imshow(torch_result)
axes[2].set_title(f"Convolution PyTorch ({kernel_name})")
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Vérifier si les résultats sont similaires
difference = np.abs(manual_result - torch_result)
print(f"Différence maximale: {difference.max()}")
print(f"Différence moyenne: {difference.mean()}")

## 5. Entraînement d'une Couche de Convolution avec PyTorch

Nous allons maintenant créer des données synthétiques et entraîner une couche de convolution pour détecter des motifs spécifiques.

### 5.1 Génération de Données Synthétiques

Générons deux classes d'images:
1. Images avec des lignes verticales
2. Images avec des lignes horizontales

In [ ]:
def generate_synthetic_data(num_samples=1000, image_size=28, noise_level=0.1):
    """Génère des données synthétiques: lignes verticales (classe 0) et horizontales (classe 1)."""
    X = np.zeros((num_samples, 1, image_size, image_size), dtype=np.float32)
    y = np.zeros(num_samples, dtype=np.int64)
    
    # Pour chaque échantillon
    for i in range(num_samples):
        # Décider aléatoirement de la classe
        class_id = i % 2  # Alterner entre classe 0 et 1
        y[i] = class_id
        
        # Générer l'image
        if class_id == 0:  # Lignes verticales
            # Choisir des positions aléatoires pour les lignes
            positions = np.random.choice(range(2, image_size-2), size=3, replace=False)
            for pos in positions:
                X[i, 0, :, pos:pos+2] = 1.0
        else:  # Lignes horizontales
            # Choisir des positions aléatoires pour les lignes
            positions = np.random.choice(range(2, image_size-2), size=3, replace=False)
            for pos in positions:
                X[i, 0, pos:pos+2, :] = 1.0
        
        # Ajouter du bruit
        X[i, 0] += noise_level * np.random.randn(image_size, image_size)
        
        # Normaliser entre 0 et 1
        X[i, 0] = np.clip(X[i, 0], 0, 1)
    
    return X, y

# Générer des données d'entraînement et de test
X_train, y_train = generate_synthetic_data(num_samples=1000)
X_test, y_test = generate_synthetic_data(num_samples=200)

# Convertir en tenseurs PyTorch
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test)

# Créer des datasets et dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Visualiser quelques exemples
def show_examples(X, y, num_examples=5):
    fig, axes = plt.subplots(2, num_examples, figsize=(15, 6))
    
    for cls in range(2):
        # Trouver les indices des exemples de cette classe
        indices = np.where(y == cls)[0][:num_examples]
        
        for i, idx in enumerate(indices):
            axes[cls, i].imshow(X[idx, 0])
            axes[cls, i].set_title(f"Classe {cls}")
            axes[cls, i].axis('off')
    
    plt.tight_layout()
    plt.show()

show_examples(X_train, y_train)

### 5.2 Définition du Modèle CNN Simple

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Une seule couche de convolution avec 2 filtres 3x3
        self.conv = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, padding=1)
        # Couche de pooling pour réduire la dimension
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Couche entièrement connectée pour la classification
        self.fc = nn.Linear(2 * 14 * 14, 2)  # 2 filtres, image 14x14 après pooling
    
    def forward(self, x):
        # Convolution et activation
        x = F.relu(self.conv(x))
        # Pooling
        x = self.pool(x)
        # Aplatir
        x = x.view(-1, 2 * 14 * 14)
        # Classification
        x = self.fc(x)
        return x
    
    def get_filters(self):
        """Récupère les filtres appris."""
        return self.conv.weight.data.cpu().numpy()

# Créer le modèle
model = SimpleCNN()
print(model)

### 5.3 Entraînement du Modèle

In [ ]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fonction d'entraînement
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    losses = []
    accuracies = []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            # Réinitialiser les gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass et optimisation
            loss.backward()
            optimizer.step()
            
            # Statistiques
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / total
        epoch_acc = correct / total
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")
    
    return losses, accuracies

# Fonction d'évaluation
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f"Accuracy de test: {accuracy:.4f}")
    return accuracy

# Entraîner le modèle
losses, accuracies = train(model, train_loader, criterion, optimizer, num_epochs=10)

# Évaluer sur les données de test
test_accuracy = evaluate(model, test_loader)

### 5.4 Visualisation des Filtres Appris

In [ ]:
# Visualiser les filtres appris
filters = model.get_filters()
print(f"Forme des filtres: {filters.shape}")

# Afficher les filtres
fig, axes = plt.subplots(2, 1, figsize=(8, 8))
for i in range(2):
    filter_img = filters[i, 0]
    axes[i].imshow(filter_img)
    axes[i].set_title(f"Filtre {i+1}")
    axes[i].axis('off')
    
    # Afficher les valeurs numériques
    for y in range(filter_img.shape[0]):
        for x in range(filter_img.shape[1]):
            axes[i].text(x, y, f"{filter_img[y, x]:.2f}", 
                         ha='center', va='center', 
                         color='white' if abs(filter_img[y, x]) > 0.5 else 'black')

plt.tight_layout()
plt.show()

### 5.5 Visualisation des Feature Maps

In [ ]:
# Créer une version du modèle pour extraire les feature maps
class FeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(
            original_model.conv,
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.features(x)

feature_extractor = FeatureExtractor(model)

# Visualiser les feature maps pour quelques exemples de test
def visualize_feature_maps(model, X, y, num_examples=2):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    with torch.no_grad():
        for cls in range(2):
            # Trouver un exemple de cette classe
            idx = np.where(y == cls)[0][0]
            sample = torch.from_numpy(X[idx:idx+1])
            
            # Obtenir les feature maps
            feature_maps = model(sample).squeeze().numpy()
            
            # Afficher l'image d'entrée
            axes[cls, 0].imshow(X[idx, 0])
            axes[cls, 0].set_title(f"Entrée (Classe {cls})")
            axes[cls, 0].axis('off')
            
            # Afficher les feature maps
            for i in range(2):
                axes[cls, i+1].imshow(feature_maps[i])
                axes[cls, i+1].set_title(f"Feature Map {i+1}")
                axes[cls, i+1].axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_feature_maps(feature_extractor, X_test, y_test)

## Conclusion

Ce notebook nous a permis d'explorer en profondeur les convolutions:

1. Nous avons implémenté manuellement l'opération de convolution 2D
2. Nous avons appliqué différents filtres sur des images MNIST
3. Nous avons comparé notre implémentation avec celle de PyTorch
4. Nous avons créé un modèle simple CNN et l'avons entraîné sur des données synthétiques
5. Nous avons visualisé les filtres appris et les feature maps générées

Les convolutions sont à la base de nombreuses applications en vision par ordinateur. Elles permettent d'extraire des caractéristiques locales des images, ce qui est particulièrement utile pour des tâches telles que la classification d'images, la détection d'objets, et la segmentation.